In [290]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import the necessary libraries

In [291]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset

In [292]:
df_books = pd.read_csv('/content/drive/MyDrive/Dataset/Books.csv', on_bad_lines='skip', encoding='latin-1')
df_users = pd.read_csv('/content/drive/MyDrive/Dataset/Users.csv', on_bad_lines='skip', encoding='latin-1')
df_ratings = pd.read_csv('/content/drive/MyDrive/Dataset/Ratings.csv', on_bad_lines='skip', encoding='latin-1')

<ipython-input-292-4080cd7f38cf>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv('/content/drive/MyDrive/Dataset/Books.csv', on_bad_lines='skip', encoding='latin-1')


In [293]:
# what are the data types of all the columns that are present
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [294]:
df_books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [295]:
# Lets remane some wierd columns name
df_books.rename(columns={"Book-Title":'title',
                      'Book-Author':'author',
                     "Year-Of-Publication":'year',
                     "Publisher":"publisher",
                     "Image-URL-L":"image_url"},inplace=True)

In [296]:
df_books.head()

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [297]:
df_books.iloc[237]['image_url']

'http://images.amazon.com/images/P/0671027387.01.LZZZZZZZ.jpg'

In [298]:
# Now load the users dataframe
df_users = pd.read_csv('/content/drive/MyDrive/Dataset/Users.csv', on_bad_lines='skip', encoding='latin-1')
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [299]:
# what are the data types of all the columns that are present
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [300]:
# Lets remane some wierd columns name
df_users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

In [301]:
df_users.tail()

,user_id,location,age
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN
278857,278858,"dublin, n/a, ireland",NaN


In [302]:
# Now load the ratings dataframe
df_ratings = pd.read_csv('/content/drive/MyDrive/Dataset/Ratings.csv', on_bad_lines='skip', encoding='latin-1')
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [303]:
# what are the data types of all the columns that are present
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [304]:
# Lets remane some wierd columns name
df_ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)

In [305]:
df_ratings.tail()

,user_id,ISBN,rating
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10
1149779,276723,05162443314,8


In [306]:
# Shape of the dataframe
print(df_books.shape, df_users.shape, df_ratings.shape, sep='\n')


(271360, 8)
(278858, 3)
(1149780, 3)


In [307]:
df_ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

#### Check for missing values


In [309]:
df_books.isnull().sum()

ISBN           0
title          0
author         2
year           0
publisher      2
Image-URL-S    0
Image-URL-M    0
image_url      3
dtype: int64

In [310]:
df_users.isnull().sum()

user_id          0
location         0
age         110762
dtype: int64

In [311]:
df_ratings.isnull().sum()

user_id    0
ISBN       0
rating     0
dtype: int64

In [312]:
# Check the duplicates values
print(df_books.duplicated().sum())
print(df_users.duplicated().sum())
print(df_ratings.duplicated().sum())

0
0
0


In [313]:
# Handling missing values
# replacing the null valuess with null string
df_books['publisher'].fillna('', inplace=True)
df_books['author'].fillna('', inplace=True)
df_books['image_url'].fillna('', inplace=True)

# replacing the null valuess with median value of age
df_users['age'].fillna(df_users['age'].median(), inplace=True)

In [314]:
print(df_books.isnull().sum())
print('===========')
print(df_users.isnull().sum())
print('===========')
df_ratings.isnull().sum()

ISBN           0
title          0
author         0
year           0
publisher      0
Image-URL-S    0
Image-URL-M    0
image_url      0
dtype: int64
user_id     0
location    0
age         0
dtype: int64


user_id    0
ISBN       0
rating     0
dtype: int64

### Popularity Based Recommender System


In [315]:
ratings_with_books = df_ratings.merge(df_books, on='ISBN')

In [316]:
num_rating_df = ratings_with_books.groupby('title')['rating'].count().reset_index()
num_rating_df.rename(columns={'rating':'num_of_ratings'},inplace=True)
num_rating_df.head(8)

,title,num_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
5,Clifford Visita El Hospital (Clifford El Gran...,1
6,Dark Justice,1
7,Deceived,2


In [317]:
average_rating_df = pd.DataFrame(ratings_with_books.groupby('title')['rating'].mean()).reset_index()
average_rating_df.rename(columns={'rating':'average_rating'},inplace=True)

In [318]:
average_rating_df.head(8)

,title,average_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00
5,Clifford Visita El Hospital (Clifford El Gran...,0.00
6,Dark Justice,10.00
7,Deceived,0.00


In [319]:
popular_df = num_rating_df.merge(average_rating_df, on='title')
popular_df

,title,num_of_ratings,average_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Ã?lpiraten.,2,0.000000
241067,Ã?Ã?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Ã?sterlich leben.,1,7.000000
241069,Ã?Ã?stlich der Berge.,3,2.666667


In [320]:
popular_df = popular_df[popular_df['num_of_ratings']>=250].sort_values('average_rating',ascending=False).head(50)

In [321]:
popular_df = popular_df.merge(df_books,on='title').drop_duplicates('title')[['title','author','image_url','num_of_ratings','average_rating']]

In [322]:
popular_df

,title,author,image_url,num_of_ratings,average_rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339703.0...,368,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,http://images.amazon.com/images/P/059035342X.0...,575,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339711.0...,260,4.880769
39,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,510,4.700000


### Collaborative filtering based Recommendation System


In [323]:
# Filtering out those users, who have Given ratings to books more than 150 Times
x = ratings_with_books.groupby('user_id').count()['rating'] > 150


In [324]:
x[x].shape

(1115,)

In [325]:
imp_users = x[x].index

In [326]:
filtered_rating = ratings_with_books[ratings_with_books['user_id'].isin(imp_users)]

In [327]:
y = filtered_rating.groupby('title').count()['rating']>=50

In [328]:
famous_books = y[y].index

In [329]:
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road',
       'A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action',
       ...
       'Without Remorse', 'Wizard and Glass (The Dark Tower, Book 4)',
       'Women Who Run with the Wolves',
       'Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players',
       'Wuthering Heights', 'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=868)

In [330]:
final_ratings = filtered_rating[filtered_rating['title'].isin(famous_books)]


In [331]:
final_ratings

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-S,Image-URL-M,image_url
63,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
65,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
66,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
67,8253,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
69,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
...,...,...,...,...,...,...,...,...,...,...
1027923,269566,0670809381,0,Echoes,Maeve Binchy,1986,Penguin USA,http://images.amazon.com/images/P/0670809381.0...,http://images.amazon.com/images/P/0670809381.0...,http://images.amazon.com/images/P/0670809381.0...
1028777,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...,http://images.amazon.com/images/P/0440910927.0...
1029070,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...,http://images.amazon.com/images/P/B0001PIOX4.0...
1030363,274808,0449701913,0,Homecoming,Cynthia Voigt,1983,Fawcett Books,http://images.amazon.com/images/P/0449701913.0...,http://images.amazon.com/images/P/0449701913.0...,http://images.amazon.com/images/P/0449701913.0...


In [332]:
final_ratings.shape

(75201, 10)

### Creating Pivot Table of UsersID vs Books and values are Ratings.



In [333]:
df_pivot = final_ratings.pivot_table(index='title',columns='user_id',values='rating')

In [334]:
df_pivot.fillna(0,inplace=True)

In [335]:
df_pivot.shape

(868, 1110)

In [336]:
df_pivot

user_id,254,1733,1903,2033,2110,2276,2766,2891,2977,3363,...,274301,274308,274808,275970,276680,277427,277478,277639,278188,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Using cosine_similarity to calculate the distance of the values of one book with another in a 2-D space (euclidean distance)

In [337]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(df_pivot)

In [338]:
similarity_scores.shape

(868, 868)

In [339]:
similarity_scores

array([[1.        , 0.07980265, 0.00974903, ..., 0.09658948, 0.06448266,
        0.03549567],
       [0.07980265, 1.        , 0.25233692, ..., 0.06000464, 0.14873547,
        0.1435961 ],
       [0.00974903, 0.25233692, 1.        , ..., 0.03769791, 0.04493654,
        0.09205667],
       ...,
       [0.09658948, 0.06000464, 0.03769791, ..., 1.        , 0.06439035,
        0.01670749],
       [0.06448266, 0.14873547, 0.04493654, ..., 0.06439035, 1.        ,
        0.09936127],
       [0.03549567, 0.1435961 , 0.09205667, ..., 0.01670749, 0.09936127,
        1.        ]])

### Fecting the Indexes of Books which are at the closest distance to the Book(input)

In [340]:
def book_recommendation(book_name):
    # index fetch
    index = np.where(df_pivot.index == book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:11]

    data = []
    for i in similar_items:
        item = []
        temp_df = df_books[df_books['title'] == df_pivot.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('title')['title'].values))
        item.extend(list(temp_df.drop_duplicates('title')['author'].values))
        item.extend(list(temp_df.drop_duplicates('title')['image_url'].values))
        item.append(i[1])  # Add the similarity score to the item list

        data.append(item)

    return data

In [341]:
book_recommendation('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.LZZZZZZZ.jpg',
  0.2231165612580781],
 ['Lord of the Flies',
  'William Gerald Golding',
  'http://images.amazon.com/images/P/0399501487.01.LZZZZZZZ.jpg',
  0.2221767355652058],
 ["Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death",
  'Kurt Vonnegut',
  'http://images.amazon.com/images/P/0440180295.01.LZZZZZZZ.jpg',
  0.2215865995231868],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.LZZZZZZZ.jpg',
  0.2204104698524294],
 ['The Catcher in the Rye',
  'J.D. Salinger',
  'http://images.amazon.com/images/P/0316769487.01.LZZZZZZZ.jpg',
  0.2030851694081133],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.LZZZZZZZ.jpg',
  0.20184379038872902],
 ['Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players',
  'Stefan Fatsis',
  'http://images.amazon.co

In [342]:
book_recommendation('Brave New World')

[["Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death",
  'Kurt Vonnegut',
  'http://images.amazon.com/images/P/0440180295.01.LZZZZZZZ.jpg',
  0.2112853613544144],
 ['1984',
  'George Orwell',
  'http://images.amazon.com/images/P/0451524934.01.LZZZZZZZ.jpg',
  0.20184379038872902],
 ['Vittorio the Vampire: New Tales of the Vampires',
  'Anne Rice',
  'http://images.amazon.com/images/P/0345422392.01.LZZZZZZZ.jpg',
  0.1971977233019704],
 ['Five Quarters of the Orange',
  'Joanne Harris',
  'http://images.amazon.com/images/P/0060958022.01.LZZZZZZZ.jpg',
  0.19150773367806428],
 ['Love in the Time of Cholera (Penguin Great Books of the 20th Century)',
  'Gabriel Garcia Marquez',
  'http://images.amazon.com/images/P/0140119906.01.LZZZZZZZ.jpg',
  0.17975380483975534],
 ['To Kill a Mockingbird',
  'Harper Lee',
  'http://images.amazon.com/images/P/0446310786.01.LZZZZZZZ.jpg',
  0.17886370026834947],
 ['How to Be Good',
  'Nick Hornby',
  'http://images.amazon.com/images/

## Convert pivot table to sparse matrix

In [343]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(df_pivot)


In [344]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

## Build the KNN Model

In [345]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')
model.fit(book_sparse)


NearestNeighbors(algorithm='brute')

In [346]:
distance, suggestion = model.kneighbors(df_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )


In [347]:
print(distance)


[[ 0.         38.78143886 39.16631206 39.28103868 39.68626967 39.83716857]]


In [348]:
print(suggestion)

[[237 377 227 427 173 126]]


In [349]:
df_pivot.iloc[241,:]

user_id
254       0.0
1733      0.0
1903      0.0
2033      0.0
2110      0.0
         ... 
277427    0.0
277478    0.0
277639    0.0
278188    0.0
278418    0.0
Name: Fortune's Rocks: A Novel, Length: 1110, dtype: float64

In [350]:
for i in range(len(suggestion)):
    print(df_pivot.index[suggestion[i]])

Index(['Flesh and Blood', 'Malice', 'Fatal Cure', 'No Safe Place',
       'Deck the Halls (Holiday Classics)', 'Choke'],
      dtype='object', name='title')


In [351]:
#keeping books name
book_names = df_pivot.index

In [352]:
book_names[2]


'2nd Chance'

In [353]:
ids = np.where(final_ratings['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]


In [354]:
final_ratings.iloc[ids]['image_url']

'http://images.amazon.com/images/P/0439064864.01.LZZZZZZZ.jpg'

## Make recommendation with KNN save Model

In [355]:
def get_knn_recommendations_book(book_name):
    book_id = np.where(df_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(df_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors=6)

    print(f"You searched '{book_name}'\n")
    print("The suggestion books are: \n")

    for i in range(len(suggestion)):
        books = df_pivot.index[suggestion[i]]
        distances = distance[i]
        for j, d in zip(books, distances):
            if j != book_name:
                print(f"{j} (Distance: {d:.4f})")


In [356]:
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
get_knn_recommendations_book(book_name)

You searched 'Harry Potter and the Chamber of Secrets (Book 2)'

The suggestion books are: 

Harry Potter and the Goblet of Fire (Book 4) (Distance: 70.3908)
Harry Potter and the Prisoner of Azkaban (Book 3) (Distance: 70.5587)
Harry Potter and the Sorcerer's Stone (Book 1) (Distance: 77.5555)
Jacob Have I Loved (Distance: 81.5840)
Exclusive (Distance: 81.6024)
